In [1]:
import pandas as pd
import requests


url = "https://realtor.p.rapidapi.com/properties/v2/list-sold"

querystring = {"city":"Philadelphia","offset":"0","state_code":"PA","limit":"10000","sort":"sold_date"}

headers = {
    'x-rapidapi-key': "da433ebb47mshbd9b291c756be35p15c636jsn8abcc28ad9fc",
    'x-rapidapi-host': "realtor.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)




In [2]:
querystring2 = {"city":"Philadelphia","offset":"900","state_code":"PA","limit":"200"}


response2 = requests.request("GET", url, headers=headers, params=querystring2)

# print(response2.text)


In [3]:
jsonResponse2 = response2.json()
print(jsonResponse2['properties'][0]['property_id'])
df2 = pd.json_normalize(jsonResponse2['properties'])
print(df2.head())

O9569591064
   property_id  listing_id      prop_type             list_date  \
0  O9569591064  2925547639          condo  2021-02-05T21:55:27Z   
1  O3188647665  2921711448          condo  2020-09-24T21:16:08Z   
2  O4088664995  2925574101  single_family  2021-01-31T01:08:36Z   
3  O4082553380  2924831310          condo  2021-01-07T05:19:04Z   
4  O4503687572  2922101835          condo  2020-10-05T21:58:53Z   

            last_update  year_built  beds  baths_full  baths_half  \
0  2021-03-10T15:25:44Z      1994.0   2.0         2.0         NaN   
1  2021-03-09T12:17:40Z         NaN   1.0         1.0         NaN   
2  2021-03-10T11:24:09Z      1925.0   8.0         4.0         1.0   
3  2021-03-09T17:12:25Z      1953.0   3.0         2.0         NaN   
4  2021-03-09T14:42:33Z      1923.0   2.0         2.0         1.0   

    prop_status  ...        price_reduced_date address.lat address.county  \
0  not_for_sale  ...                       NaN         NaN            NaN   
1  not_for_sale 

In [4]:
jsonResponse = response.json()
print(jsonResponse['properties'][0]['property_id'])
df = pd.json_normalize(jsonResponse['properties'])

O4027323877


In [5]:
# print(df.head())
print(df2.shape)
print(df.shape)
df = pd.concat([df, df2]).drop_duplicates('property_id').reset_index(drop=True)
print("after concat")
print(df.shape)

(200, 73)
(200, 73)
after concat
(400, 73)


In [6]:
# page the api and gather the data
for i in range (0, 10000, 200):
    querystring2 = {"city":"Philadelphia","offset":"{}".format(i),"state_code":"PA","limit":"200"}


    response2 = requests.request("GET", url, headers=headers, params=querystring2)
    
    jsonResponse2 = response2.json()
# print(jsonResponse2['properties'][0]['property_id'])
    df2 = pd.json_normalize(jsonResponse2['properties'])
# print(df2.head())
    
    print("Prev dataframe")
    print(df2.shape)
    print("new dataframe")
#     print(df.shape)
    df = pd.concat([df, df2]).drop_duplicates('property_id').reset_index(drop=True)
    print("after concat")
    print(df.shape)

Prev dataframe
(200, 73)
new dataframe
after concat
(401, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(600, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(798, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(997, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(1095, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(1195, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(1395, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(1594, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(1794, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(1994, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(2192, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(2392, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(2589, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(2786, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(2985, 73)
Prev dataframe
(200, 73)
new dataframe
after concat
(3184, 

In [7]:
#Save ALL data
df.to_csv("All_House_Data_Raw.csv", index=False)
print(df.columns)

Index(['property_id', 'listing_id', 'prop_type', 'list_date', 'last_update',
       'year_built', 'beds', 'baths_full', 'price_reduced_date', 'baths_half',
       'prop_status', 'sold_history', 'products', 'agents', 'price',
       'rdc_web_url', 'rdc_app_url', 'baths', 'data_source_name', 'page_no',
       'rank', 'list_tracking', 'is_new_construction', 'photo_count', 'photos',
       'address.line', 'address.street_number', 'address.street',
       'address.street_suffix', 'address.unit', 'address.city',
       'address.state', 'address.state_code', 'address.postal_code',
       'address.lat', 'address.county', 'address.neighborhood_name',
       'address.neighborhoods', 'address.lon', 'mls.abbreviation', 'mls.id',
       'client_display_flags.presentation_status',
       'client_display_flags.is_showcase',
       'client_display_flags.lead_form_phone_required',
       'client_display_flags.price_change',
       'client_display_flags.is_co_broke_email',
       'client_display_flags.h

In [10]:
# Reduced data frame contains the raw data from Realtor.com that I want to use in my model
r_df = df[['last_update','address.lat','address.lon','lot_size.size','prop_type', 'year_built', 'beds','price', 'baths_full', 'baths_half', 'is_new_construction','address.postal_code', 'address.neighborhood_name', 'building_size.size', 'garage']]
print(r_df.shape)

#Remove land and multifamily property types because they contain incomplete/insufficient data
r_df = r_df[r_df.prop_type != 'land']
r_df = r_df[r_df.prop_type != 'multi_family']
r_df['garage'] = r_df['garage'].fillna(0)
r_df['baths_half'] = r_df['baths_half'].fillna(0)
r_df['address.postal_code']=r_df['address.postal_code'].astype(int)

print(r_df.shape)
print(r_df.describe())
print(r_df.info())

(9893, 15)
(5825, 15)
       address.lat  address.lon  lot_size.size   year_built         beds  \
count  5125.000000  5125.000000   5.088000e+03  5544.000000  4843.000000   
mean     40.003997   -75.136333   1.892027e+04  1945.088564     2.978319   
std       0.056378     0.092701   1.152977e+06    34.483738     0.938057   
min      39.891611   -79.561390   1.280000e+02  1750.000000     0.000000   
25%      39.952616   -75.180802   9.307500e+02  1920.000000     3.000000   
50%      40.005017   -75.152858   1.440000e+03  1939.000000     3.000000   
75%      40.048620   -75.079773   2.593500e+03  1962.000000     3.000000   
max      40.993561   -74.964251   8.224128e+07  2021.000000     9.000000   

              price   baths_full   baths_half  address.postal_code  \
count  5.824000e+03  4876.000000  5825.000000          5825.000000   
mean   3.259755e+05     1.565217     0.362918         19131.456652   
std    3.090839e+05     0.726673     0.522632            14.490626   
min    1.1500

In [3]:
import pandas as pd
# Reduced data frame contains the raw data from Realtor.com that I want to use in my model
df = pd.read_csv('All_House_Data_Raw.csv')
reduced_df = df[['list_date','address.lat','address.lon','lot_size.size','prop_type', 'year_built', 'beds','price', 'baths_full', 'baths_half', 'is_new_construction','address.postal_code', 'address.neighborhood_name', 'building_size.size', 'garage']]

# print(reduced_df.head())

# reduced_df.info()
reduced_df = reduced_df[reduced_df.prop_type != 'land']
reduced_df = reduced_df[reduced_df.prop_type != 'multi_family']
reduced_df['garage'] = reduced_df['garage'].fillna(0)
reduced_df['baths_half'] = reduced_df['baths_half'].fillna(0)
reduced_df['address.postal_code']=reduced_df['address.postal_code'].astype(int)
pd.set_option('display.max_rows', reduced_df.shape[0]+1)
print(reduced_df.head)

print(reduced_df.isnull().sum())



<bound method NDFrame.head of                  list_date  address.lat  address.lon  lot_size.size  \
0     2021-03-05T17:49:21Z    39.926237   -75.176307          700.0   
1     2021-02-17T20:47:19Z    40.069313   -75.075879         3171.0   
2     2021-02-05T19:01:14Z    40.001261   -75.117655          602.0   
3     2021-01-26T16:59:31Z    39.987430   -75.100497          909.0   
4     2020-11-20T16:02:24Z    39.937969   -75.179954         1044.0   
5     2021-01-07T17:22:23Z    39.932317   -75.158763          588.0   
7     2020-10-02T15:48:45Z    39.919608   -75.166762          960.0   
8     2020-08-25T16:53:25Z    39.924942   -75.165559          881.0   
9     2020-12-04T00:56:07Z    39.945303   -75.157182            NaN   
10    2020-08-26T14:41:54Z    40.042148   -75.075913         1260.0   
11    2021-01-28T05:17:31Z    40.018977   -75.209557         1241.0   
12    2020-10-22T20:29:48Z    39.942765   -75.144045            NaN   
13    2020-10-20T20:33:45Z    39.956137   -75.2

In [4]:

# Read unemployment rate data from excel
df_unemployed = pd.read_excel('unemployment_data_Philly.xlsx')
# print(df_unemployed)

# Read median income data from excel
df_median_income = pd.read_excel('median_income_Philly.xlsx')
# print(df_median_income)

# merge the two datasets based on zip code
merged_inner = pd.merge(left=df_unemployed, right=df_median_income, left_on='Zip Code', right_on='Zip Code')
print("Merged Table:")
# print(merged_inner)
# r_df.info()

# left join the house sale data with zip code data
merged_left = pd.merge(left=reduced_df, right=merged_inner, how='left', left_on='address.postal_code', right_on='Zip Code')
print(merged_left.head())

Merged Table:
              list_date  address.lat  address.lon  lot_size.size prop_type  \
0  2021-03-05T17:49:21Z    39.926237   -75.176307          700.0     condo   
1  2021-02-17T20:47:19Z    40.069313   -75.075879         3171.0     condo   
2  2021-02-05T19:01:14Z    40.001261   -75.117655          602.0     condo   
3  2021-01-26T16:59:31Z    39.987430   -75.100497          909.0     condo   
4  2020-11-20T16:02:24Z    39.937969   -75.179954         1044.0     condo   

   year_built  beds     price  baths_full  baths_half  is_new_construction  \
0      1920.0   3.0  160000.0         1.0         0.0                False   
1      1957.0   3.0  265000.0         1.0         1.0                False   
2      1920.0   2.0   25000.0         1.0         0.0                False   
3      1920.0   3.0  265000.0         1.0         1.0                False   
4      2020.0   3.0  742500.0         3.0         1.0                 True   

   address.postal_code             address.neigh

In [5]:
merged_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5825 entries, 0 to 5824
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   list_date                  4157 non-null   object 
 1   address.lat                5125 non-null   float64
 2   address.lon                5125 non-null   float64
 3   lot_size.size              5088 non-null   float64
 4   prop_type                  5815 non-null   object 
 5   year_built                 5544 non-null   float64
 6   beds                       4843 non-null   float64
 7   price                      5824 non-null   float64
 8   baths_full                 4876 non-null   float64
 9   baths_half                 5825 non-null   float64
 10  is_new_construction        5825 non-null   bool   
 11  address.postal_code        5825 non-null   int32  
 12  address.neighborhood_name  5121 non-null   object 
 13  building_size.size         5777 non-null   float

In [6]:

# export reduced dataframe to csv for backup
merged_left.to_csv("reduced_housing_data_noland_or_mfh.csv")